In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [71]:
PL_19_20_df = pd.read_csv('2019_2020_Player_Stats.csv', skiprows=1)
PL_18_19_df = pd.read_csv('2018_2019_Player_Stats.csv', skiprows=1)
PL_17_18_df = pd.read_csv('2017_2018_Player_Stats.csv', skiprows=1)

In [5]:
PL_19_20_df.head()

,Rk,Player▲,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
0,1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,20,20,1714.0,...,0.11,1.8,1.0,1.6,0.09,0.08,0.18,0.05,0.14,Matches
1,2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19.0,2000.0,25,25,2250.0,...,0.00,0.5,0.5,3.4,0.02,0.14,0.16,0.02,0.16,Matches
2,3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,25,23,1945.0,...,0.74,12.2,12.2,2.3,0.56,0.11,0.67,0.56,0.67,Matches
3,4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,21,8,712.0,...,0.25,1.9,1.9,0.9,0.24,0.11,0.35,0.24,0.35,Matches
4,5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,...,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


### Clean up datasets

In [15]:
#clean up 19-20
PL_19_20_df = PL_19_20_df.rename(columns={"Player▲": "Player"})
PL_19_20_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
0,1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,20,20,1714.0,...,0.11,1.8,1.0,1.6,0.09,0.08,0.18,0.05,0.14,Matches
1,2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19.0,2000.0,25,25,2250.0,...,0.00,0.5,0.5,3.4,0.02,0.14,0.16,0.02,0.16,Matches
2,3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,25,23,1945.0,...,0.74,12.2,12.2,2.3,0.56,0.11,0.67,0.56,0.67,Matches
3,4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,21,8,712.0,...,0.25,1.9,1.9,0.9,0.24,0.11,0.35,0.24,0.35,Matches
4,5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,...,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [18]:
player_name_split = PL_19_20_df['Player'].str.split('\\')
player_name_split

0        [Patrick van Aanholt, Patrick-van-Aanholt]
1                          [Max Aarons, Max-Aarons]
2                    [Tammy Abraham, Tammy-Abraham]
3                            [Che Adams, Che-Adams]
4                                  [Adrián, Adrian]
                           ...                     
483                  [Wilfried Zaha, Wilfried-Zaha]
484    [Christoph Zimmermann, Christoph-Zimmermann]
485      [Oleksandr Zinchenko, Oleksandr-Zinchenko]
486                        [Kurt Zouma, Kurt-Zouma]
487                        [Mesut Özil, Mesut-Ozil]
Name: Player, Length: 488, dtype: object

In [19]:
player_names = player_name_split.map(lambda _:_[0])
player_names

0       Patrick van Aanholt
1                Max Aarons
2             Tammy Abraham
3                 Che Adams
4                    Adrián
               ...         
483           Wilfried Zaha
484    Christoph Zimmermann
485     Oleksandr Zinchenko
486              Kurt Zouma
487              Mesut Özil
Name: Player, Length: 488, dtype: object

In [22]:
PL_19_20_df['Player'] = player_names
PL_19_20_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches,Player Name
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,20,20,1714.0,...,1.8,1.0,1.6,0.09,0.08,0.18,0.05,0.14,Matches,Patrick van Aanholt
1,2,Max Aarons,eng ENG,DF,Norwich City,19.0,2000.0,25,25,2250.0,...,0.5,0.5,3.4,0.02,0.14,0.16,0.02,0.16,Matches,Max Aarons
2,3,Tammy Abraham,eng ENG,FW,Chelsea,21.0,1997.0,25,23,1945.0,...,12.2,12.2,2.3,0.56,0.11,0.67,0.56,0.67,Matches,Tammy Abraham
3,4,Che Adams,eng ENG,FW,Southampton,23.0,1996.0,21,8,712.0,...,1.9,1.9,0.9,0.24,0.11,0.35,0.24,0.35,Matches,Che Adams
4,5,Adrián,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches,Adrián


In [24]:
PL_19_20_df = PL_19_20_df.drop(columns={'Player Name'})
PL_19_20_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,20,20,1714.0,...,0.11,1.8,1.0,1.6,0.09,0.08,0.18,0.05,0.14,Matches
1,2,Max Aarons,eng ENG,DF,Norwich City,19.0,2000.0,25,25,2250.0,...,0.00,0.5,0.5,3.4,0.02,0.14,0.16,0.02,0.16,Matches
2,3,Tammy Abraham,eng ENG,FW,Chelsea,21.0,1997.0,25,23,1945.0,...,0.74,12.2,12.2,2.3,0.56,0.11,0.67,0.56,0.67,Matches
3,4,Che Adams,eng ENG,FW,Southampton,23.0,1996.0,21,8,712.0,...,0.25,1.9,1.9,0.9,0.24,0.11,0.35,0.24,0.35,Matches
4,5,Adrián,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,...,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [50]:
nation_split = PL_19_20_df['Nation'].str.split(' ', 1, expand=True)

,0,1
0,nl,NED
1,eng,ENG
2,eng,ENG
3,eng,ENG
4,es,ESP
...,...,...
483,ci,CIV
484,de,GER
485,ua,UKR
486,fr,FRA


In [55]:
nation_split.rename(columns={0: 'A', 1: 'B'}, inplace=True)

In [57]:
nation_split.drop(columns={'A'})

,B
0,NED
1,ENG
2,ENG
3,ENG
4,ESP
...,...
483,CIV
484,GER
485,UKR
486,FRA


In [60]:
PL_19_20_df['Nation'] = nation_split['B']

In [61]:
PL_19_20_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
0,1,Patrick van Aanholt,NED,DF,Crystal Palace,28.0,1990.0,20,20,1714.0,...,0.11,1.8,1.0,1.6,0.09,0.08,0.18,0.05,0.14,Matches
1,2,Max Aarons,ENG,DF,Norwich City,19.0,2000.0,25,25,2250.0,...,0.00,0.5,0.5,3.4,0.02,0.14,0.16,0.02,0.16,Matches
2,3,Tammy Abraham,ENG,FW,Chelsea,21.0,1997.0,25,23,1945.0,...,0.74,12.2,12.2,2.3,0.56,0.11,0.67,0.56,0.67,Matches
3,4,Che Adams,ENG,FW,Southampton,23.0,1996.0,21,8,712.0,...,0.25,1.9,1.9,0.9,0.24,0.11,0.35,0.24,0.35,Matches
4,5,Adrián,ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,...,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [79]:
PL_19_20_df.drop(columns=["Rk"], axis=1, inplace=True)
PL_19_20_df.drop(columns=["Matches"], axis=1, inplace=True)

In [74]:
#clean up 18-19
PL_18_19_df = PL_18_19_df.rename(columns={"Player▲": "Player"})
player_name_split = PL_18_19_df['Player'].str.split('\\')
player_names = player_name_split.map(lambda _:_[0])
PL_18_19_df['Player'] = player_names
nation_split = PL_18_19_df['Nation'].str.split(' ', 1, expand=True)

,0,1
0,nl,NED
1,ar,ARG
2,nl,NED
3,eng,ENG
4,be,BEL
...,...,...
503,cm,CMR
504,it,ITA
505,ua,UKR
506,dk,DEN


In [78]:
nation_split.rename(columns={0: 'A', 1: 'B'}, inplace=True)
nation_split.drop(columns={'A'})
PL_18_19_df['Nation'] = nation_split['B']
PL_18_19_df.drop(columns=["Rk"], axis=1, inplace=True)
PL_18_19_df.drop(columns=["Matches"], axis=1, inplace=True)
PL_18_19_df.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,...,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA
0,Patrick van Aanholt,NED,DF,Crystal Palace,27.0,1990.0,36,36,3198.0,3,...,0.08,0.14,2.9,2.9,4.0,0.08,0.11,0.19,0.08,0.19
1,Sergio Agüero,ENG,FW,Manchester City,30.0,1988.0,33,31,2480.0,21,...,0.69,0.98,20.3,18.8,5.2,0.74,0.19,0.93,0.68,0.87
2,Nathan Aké,ENG,DF,Bournemouth,23.0,1995.0,38,38,3412.0,4,...,0.11,0.11,3.5,3.5,0.7,0.09,0.02,0.11,0.09,0.11
3,Marc Albrighton,SCO,FWMF,Leicester City,28.0,1989.0,27,18,1626.0,2,...,0.11,0.22,1.0,1.0,2.9,0.05,0.16,0.21,0.05,0.21
4,Toby Alderweireld,ESP,DF,Tottenham,29.0,1989.0,34,33,2923.0,0,...,0.00,0.00,1.3,1.3,0.5,0.04,0.02,0.05,0.04,0.05


In [81]:
#clean up17-18
PL_17_18_df = PL_17_18_df.rename(columns={"Player▲": "Player"})
player_name_split = PL_17_18_df['Player'].str.split('\\')
player_names = player_name_split.map(lambda _:_[0])
PL_17_18_df['Player'] = player_names
nation_split = PL_17_18_df['Nation'].str.split(' ', 1, expand=True)
nation_split.rename(columns={0: 'A', 1: 'B'}, inplace=True)
nation_split.drop(columns={'A'})
PL_17_18_df['Nation'] = nation_split['B']
PL_17_18_df.drop(columns=["Rk"], axis=1, inplace=True)
PL_17_18_df.drop(columns=["Matches"], axis=1, inplace=True)
PL_17_18_df.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,...,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA
0,Patrick van Aanholt,NED,DF,Crystal Palace,26,1990,28,25,2184,5,...,0.21,0.25,3.4,3.4,1.7,0.14,0.07,0.21,0.14,0.21
1,Rolando Aarons,ENG,MFFW,Newcastle Utd,21,1995,4,1,139,0,...,0.00,0.00,0.2,0.2,0.0,0.12,0.00,0.12,0.12,0.12
2,Tammy Abraham,ENG,FWMF,Swansea City,19,1997,31,15,1726,6,...,0.31,0.37,6.4,6.4,0.8,0.33,0.04,0.38,0.33,0.38
3,Charlie Adam,SCO,MF,Stoke City,31,1985,11,5,411,0,...,0.00,0.00,1.4,0.6,1.4,0.30,0.32,0.62,0.13,0.45
4,Adrián,ESP,GK,West Ham,30,1987,19,19,1710,0,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00


### Machine learning model